# Готовим данные для ARTM

В этом блокноте готовим данные, взятые из контакта для ARTM. Библиотека поддерживает [конкретные виды входа.](http://docs.bigartm.org/en/stable/tutorials/datasets.html) Будем перегонять их в **vowpal wabbit** формат, как наиболее хайповый. В нашей статье всё хайповое и трендовое. Также будем чистить данные от всякой мерзости и лемматизировать. 

In [17]:
import pandas as pd              # Пакет для работы с таблицами
import numpy as np               # Пакет для работы с векторами и матрицами

# Пакет для красивых циклов. При желании его можно отключить. 
# Тогда из всех циклов придётся  удалять команду tqdm_notebook.
from tqdm import tqdm_notebook   # подробнее: https://github.com/tqdm/tqdm

import re      # пакет для работы с регулярными выражениями 
import pickle  # пакет для подгрузки данных специфического для питона формата

# работа со временем
from datetime import datetime

# Токенизатор
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')

# Лемматизатор (приводит слова к нормальной форме)
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

# Список стоп-слов
from nltk.corpus import stopwords
stop_ru = stopwords.words('russian')
stop_en = stopwords.words('english')
stop = stop_ru + stop_en

# Для распараллеливания кода
from joblib import Parallel, delayed

# Для перемешивания
from random import shuffle

In [2]:
# Путь к данным
path = '../data/'

# жанры 
music = ['classic', 'rap', 'shanson', 'jazz']

In [3]:
def dataLoad(music_name, path = path, data_type='group_comments'):
    """ 
        Подгружает что-нибудь эдакое
        music_name: string
            название жанра
        path: string
            путь до директории с данными
        data_type
            суффикс файла (group_comments - комментарии, users_wall, ...) 
    """
    # подгружаем данные 
    with open(path + music_name + '_' + data_type, 'rb') as f:
        df = pickle.load(f)
    return df

In [4]:
def dataLoadv2(music_name, path=path, data_type='group_comments'):
    """ 
        Подгружает что-нибудь эдакое
        music_name: string
            название жанра
        path: string
            путь до директории с данными
        data_type
            суффикс файла (group_comments - комментарии, users_wall, ...) 
    """
    # находим все файлы директории
    files = os.listdir(path='{}/{}/'.format(path, music_name))
    files = [f for f in files if data_type in f]
    
    data = {}
    # подгружаем данные 
    for file in files:
        group_name = file.split(data_type+'_')[-1]
        with open('{}/{}/{}'.format(path, music_name, file), 'rb') as f:
            df = pickle.load(f)
        data[group_name] = df
    
    return data

In [5]:
def htmlStrip(text):
    """ 
        Возвращает текст, очещенный от html тэгов
        text: string
            Текст поста 
    """
    
    return re.sub('<[^<]+?>', '', str(text)) 


def tokenLemma(text, stop = stop):
    """ 
        Возвращает список слов, очищенный от тэгов, пролемматизированный и без стоп слов
        text: string
            Текст поста
        
        parameters: list 
            stop: список из стоп-слов, example: ['политик', 'выбирать']
    """
    
    # понижение регистра, очистка от тэгов, токенизация
    out_1 = tokenizer.tokenize(htmlStrip(text.lower()))
    # лемматизация и очистка от стоп-слов, ещё выкидывает слова короче 3 символов
    out_2 = [morph.parse(tx)[0].normal_form for tx in out_1 if tx not in stop and len(tx) > 2]
    return ' '.join(out_2)

## 1. Предобработка комментариев

В конечном итоге нам нужно, чтобы каждый комментарий перегонялся в формат **vowpal wabbit:**

```
'popsa_бузова_4887563_208109_208174 |@default_class id289877716 глеб публикация какой какой число день какой коммент ольга ответить это плохой комментатор один видеть олечка ответить коммент|@likes 0 |@stickers sticker_1 sticker_2  |@audio николай басков шарманка |@video клип басков шарманка |@link плейлист киркоров |@emoji 😊😊😊'
```

Модальности: 

* имя коммента `жанр исполнитель id группы_id поста_id коммента`
* текст коммента 
* число лайков
* стикеры в комменте 
* название аудиозаписи, вложенной в коммент 
* название и описание видеозаписи, вложенной в коммент 
* ссылка на плейлист или ещё куда и её описание
* эмодзи, которые были в комменте

Именно в таком виде мы будем подавать комменты в модель для обучения. Кроме такого формата будем делать прмоежуточный: обычный массив из словарей с комментами. Его можно будет быстро отфильтровать по какому-то признаку, превратить в табличку и тп. Именно его будем использовать для сентимент-модели и матерных списков Олечки. 

In [6]:
def firstCommentPrepare(comment, style, performer):
    """
        Первичная предобработка комментария в словарик
    """
    
    # интересные вложения
    stickers = [ ]  # стикеры 
    audio = [ ]     # музыка 
    video = [ ]     # видяшки
    # photo_cnt = 0   # счётчик для фоточек с красоточками
    # doc_cnt = 0     # чаще всего это фотки либо гифки
    link = [ ]      # ссылки на плейлисты или ещё куда
        
    if isinstance(comment, str):
        print(comment)
        return None

    attach = comment.get('attachments')
    if attach is not None:
        for at in attach:
            tp = at['type']
            if tp == 'sticker':
                stickers.append(tp + '_' + str(at[tp]['sticker_id']))
            elif tp == 'audio':
                audio.append(at[tp]['artist'] + ' ' + at[tp]['title'])
            elif tp == 'video':
                # из интересных вложений есть ещё shows, comments, duration
                video.append(at[tp]['title'] + ' ' + at[tp]['description'])
            # elif tp == 'photo':
            #     photo_cnt += 1
            # elif tp == 'doc':
            #     doc_cnt += 1
            elif tp == 'link':
                link.append(at[tp]['title'] + ' ' + at[tp]['description']) 
                
    # Выбрасываем из записи все символы, кроме эмодзи и редких символов вроде иероглифов         
    emoji = re.sub('[a-яa-zA-ZА-Я -.,0-9<>:=;/_—@$^?–ё`%#*&!№…\|/\"\]\[\"\'\n\t”“’‘]','',comment['text'])
                    
    comment_infa = {     
        'music_style' : style,
        'performer' : performer,
        'comment_id' :  str(comment['group_id']) + '_' + str(comment['post_id'])  + '_' + str(comment['id']),
        'author' : str(comment['from_id']),
        'likes' : str(comment['likes']),
        'date' : datetime.utcfromtimestamp(comment['date']).strftime('%Y-%m-%d %H:%M:%S'),
        'text' : tokenLemma(comment['text']),
        'dirty_text' : comment['text'],
        'emoji' : emoji if len(emoji) > 1 else None,
        'stickers' : ' '.join(stickers) if len(stickers) > 0 else None, 
        'audio' : tokenLemma(' '.join(audio)) if len(audio) > 0 else None,
        'video' : tokenLemma(' '.join(video)) if len(video) > 0 else None,
        'link' : tokenLemma(' '.join(link)) if len(link) > 0 else None
        }
    return comment_infa


def secondCommentPrepare(comment, modality=True):
    """
    Вторичная предобработка комментариев из словаря в vw
        comment : string 
            словарик с комментарием
        modality : bool
            нужны ли модальности по видео, аудио, ссылкам и тп
    """
    # инфу про автора и дату я нигде не использую 
    comment_name = comment['music_style'] + '_' + comment['performer'] + '_' + comment['comment_id']
    likes = '|@likes {} '.format(comment['likes'])
    text = '|@default_class {}'.format(comment['text'])
    
    # базовая инфа для модели 
    comment_body = comment_name + text  + likes
    
    # дополнительные модальности 
    if modality:
        if comment['stickers']:
            comment_body = comment_body + ' |@stickers ' + comment['stickers']
        if comment['audio']:
            comment_body = comment_body + ' |@audio ' + comment['audio']
        if comment['video']:
            comment_body = comment_body + ' |@video ' + comment['video']
        if comment['link']:
            comment_body = comment_body + ' |@link ' + comment['link']
        if comment['emoji']:
            comment_body = comment_body + ' |@emoji ' + comment['emoji']
        
            
    return comment_body

In [25]:
df = dataLoadv2('jazz')
df.keys()

dict_keys(['jazz_music', 'jazz_джаз', 'best_jazz', 'e_music_jazz', 'jazz_от_тео'])

In [26]:
test = firstCommentPrepare(df['jazz_music'][10],'jazz','jazz_music')
test

{'music_style': 'jazz',
 'performer': 'jazz_music',
 'comment_id': '81892801_3140_3141',
 'author': '423170807',
 'likes': '0',
 'date': '2018-10-13 17:07:01',
 'text': 'честь назвать книга',
 'dirty_text': 'А так же в честь нее названа книга)',
 'emoji': None,
 'stickers': None,
 'audio': None,
 'video': None,
 'link': None}

In [27]:
secondCommentPrepare(test)

'jazz_jazz_music_81892801_3140_3141|@default_class честь назвать книга|@likes 0 '

In [28]:
secondCommentPrepare(test, modality=True)

'jazz_jazz_music_81892801_3140_3141|@default_class честь назвать книга|@likes 0 '

In [32]:
comments = [ ]

for name in music:
    df = dataLoadv2(name)
    
    for key in tqdm_notebook(df):
        print('Предобрабатывают',name)
        # Внимание, костыль! 
        def prCom(w):
            return firstCommentPrepare(w,name,key)
        
        n_jobs = -1 # параллелим на все ядра 
        result = Parallel(n_jobs=n_jobs)(delayed(prCom)(text) for text in tqdm_notebook(df[key]))
        comments.extend(result)
        print('Текущее число комментариев:', len(comments))

Предобрабатывают classic


Текущее число комментариев: 1084
Предобрабатывают classic


Текущее число комментариев: 1107
Предобрабатывают classic


Текущее число комментариев: 4751
Предобрабатывают classic


Текущее число комментариев: 5638
Предобрабатывают classic


...

Текущее число комментариев: 2680794
Предобрабатывают jazz


Текущее число комментариев: 2702038
Предобрабатывают jazz


Текущее число комментариев: 2725567
Предобрабатывают jazz


Текущее число комментариев: 2727492
Предобрабатывают jazz


Текущее число комментариев: 2742721


In [33]:
print(len(comments))
comments[0]

2742721


{'music_style': 'classic',
 'performer': 'inclassic',
 'comment_id': '26484100_3572_3573',
 'author': '176733397',
 'likes': '58',
 'date': '2015-11-15 20:35:30',
 'text': 'просто огромный спасибо такой чудесный подборка настроение',
 'dirty_text': 'Просто огромное спасибо за такую чудесную подборку! \nКак раз под настроение!',
 'emoji': None,
 'stickers': None,
 'audio': None,
 'video': None,
 'link': None}

In [35]:
shuffle(comments) # перемешаем комменты
comments[0]

{'music_style': 'classic',
 'performer': 'inclassic',
 'comment_id': '95470601_100826_113382',
 'author': '373874355',
 'likes': '4',
 'date': '2017-09-11 17:52:23',
 'text': 'посмотреть разнообразный сравнивать трек это просто разный человек быть знать фейс включить этот трек поверить это человек творчество это просто история однако спасибо новый альбом трек репить',
 'dirty_text': 'Вот вы только посмотрите как он разнообразен.\nЕсли сравнивать 1 трек и 2 - это просто 2 разных человека. \nЕсли бы я будучи не зная фейса и мне включили 2 этих трека, я бы не поверила, что это один человек, да.\nЕго творчество это просто как какая-то история.\nОднако спасибо за новый альбом.\n1 трек на репите.',
 'emoji': None,
 'stickers': None,
 'audio': None,
 'video': None,
 'link': None}

In [36]:
'_'.join(music)

'classic_rap_shanson_jazz'

In [37]:
# сохраним предобработанные данные 
with open('../data/{}_clean_comments'.format('_'.join(music)), 'wb') as f:
    pickle.dump(comments, f)

In [10]:
with open('../data/{}_clean_comments'.format('_'.join(music)), 'rb') as f:
    comments = pickle.load(f)

In [11]:
len(comments)

2742721

In [12]:
comments = [com for com in comments if com!=None]

In [13]:
# отфильтруем слишком короткие комменты (меньше 10 символов) 
# тут же надо будет сделать сурьёзные махинации с отбором комментов по длине и сэмплом по жанрам 
comments = [com for com in comments if len(com['text']) > 10]
len(comments)

1858741

In [14]:
n_jobs = -1 # параллелим на все ядра 
artm_result = Parallel(n_jobs=n_jobs)(delayed(secondCommentPrepare)(
    text) for text in tqdm_notebook(comments))

In [15]:
artm_result[0]

'classic_inclassic_95470601_100826_113382|@default_class посмотреть разнообразный сравнивать трек это просто разный человек быть знать фейс включить этот трек поверить это человек творчество это просто история однако спасибо новый альбом трек репить|@likes 4 '

In [16]:
# сохраняем полученное добро 
with open('../data_artm/classic_rap_shanson_jazz_comments_v1', 'a') as out:
    for item in tqdm_notebook(artm_result):
        out.write("{}\n".format(item))

In [8]:
with open('../data_artm/classic_rap_shanson_jazz_comments_v1', 'r') as f:
    comments = f.read()

In [18]:
comments[:100]

[{'music_style': 'classic',
  'performer': 'inclassic',
  'comment_id': '95470601_100826_113382',
  'author': '373874355',
  'likes': '4',
  'date': '2017-09-11 17:52:23',
  'text': 'посмотреть разнообразный сравнивать трек это просто разный человек быть знать фейс включить этот трек поверить это человек творчество это просто история однако спасибо новый альбом трек репить',
  'dirty_text': 'Вот вы только посмотрите как он разнообразен.\nЕсли сравнивать 1 трек и 2 - это просто 2 разных человека. \nЕсли бы я будучи не зная фейса и мне включили 2 этих трека, я бы не поверила, что это один человек, да.\nЕго творчество это просто как какая-то история.\nОднако спасибо за новый альбом.\n1 трек на репите.',
  'emoji': None,
  'stickers': None,
  'audio': None,
  'video': None,
  'link': None},
 {'music_style': 'classic',
  'performer': 'inclassic',
  'comment_id': '38213334_118857_118889',
  'author': '86352833',
  'likes': '0',
  'date': '2015-08-04 19:31:08',
  'text': 'дать бог мечта испол

## 2. Предобработка стенок юзеров

In [ ]:
# задел на будущее

## 3. Предобраьотка постов из групп

In [ ]:
# задел на будущее